In [1]:
import os as os
import csv as csv
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from time import time
import pandas as pd
import numpy as np
from PIL import Image
import keras.layers.advanced_activations
import scipy
import random
import matplotlib.pyplot as plt  
from scipy.stats import norm  
import keras
from keras.models import Sequential
from keras.initializers import VarianceScaling,RandomNormal
from keras.layers import Input, Dense, Dropout, Conv2D, MaxPooling2D,concatenate
from keras.layers import UpSampling2D, Reshape, Lambda, Flatten, Activation
from keras.models import Model  
from keras.optimizers import SGD, Adadelta, Adagrad,Adam
from keras import backend as K  
from keras import objectives  
from keras.utils.vis_utils import plot_model  
from keras.utils import np_utils, generic_utils
import sys 
import tensorflow as tf
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.layers import BatchNormalization
from keras.layers.normalization import BatchNormalization
import matplotlib.pyplot as plt
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import sklearn
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter
%matplotlib inline
from sklearn import manifold, datasets,cluster
from sklearn.utils import check_random_state
from sklearn import metrics

Using TensorFlow backend.


In [2]:
import SimpleITK as sitk  
import cv2 

In [3]:
normalized_data = np.loadtxt("normalized_data.txt")
original_label = np.loadtxt("original_label.txt")

In [4]:
normalized_data = np.reshape(normalized_data,(-1,512,512,1))
original_label = np.reshape(original_label,(-1,512,512,1))

In [5]:
batch_size =1
latent_dim = 80
nb_epoch = 50   
intermediate_dim_1 = 10
#intermediate_dim_2 = 300

input_img = Input(shape=(512,512,1))

conv_1 = Conv2D(8, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(input_img)
maxpool_1 = MaxPooling2D((2, 2),  padding='same')(conv_1)

conv_2 = Conv2D(8, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(maxpool_1)
maxpool_2 = MaxPooling2D((2, 2),  padding='same')(conv_2)

conv_3 = Conv2D(8, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(maxpool_2)
maxpool_3 = MaxPooling2D((2, 2),  padding='same')(conv_3)

conv_4 = Conv2D(8, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(maxpool_3)
maxpool_4 = MaxPooling2D((2, 2),  padding='same')(conv_4)

conv_5 = Conv2D(8, (3, 3), activation='tanh', padding='same',kernel_initializer='normal')(maxpool_4)
maxpool_5 = MaxPooling2D((2, 2),  padding='same')(conv_5)

#x = Conv2D(5, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(x)
#x = MaxPooling2D((2, 2),  padding='same')(x)

visual = Flatten()(maxpool_5)
h_1 = Dense(intermediate_dim_1, activation='relu')(visual)
h_4 = Dense(intermediate_dim_1,activation='tanh')(h_1)
h_5 = Dense(8*16*16,activation='relu')(h_4)
h_6 = Reshape((16,16,8))(h_5)


conv_6 = Conv2D(8, (3, 3), activation='tanh', padding='same',kernel_initializer='normal')(h_6)
upsample_6 = UpSampling2D((2, 2))(conv_6)

conv_7 = Conv2D(8, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(upsample_6)
upsample_7 = UpSampling2D((2, 2))(conv_7)

conv_8 = Conv2D(8, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(upsample_7)
upsample_8 = UpSampling2D((2, 2))(conv_8)

conv_9 = Conv2D(8, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(upsample_8)
upsample_9 = UpSampling2D((2, 2))(conv_9)

conv_10 = Conv2D(8,  (3, 3), activation='tanh',padding='same',kernel_initializer='normal')(upsample_9)
upsample_10 = UpSampling2D((2, 2))(conv_10)

decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(upsample_10)

EarlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto')


def vae_loss(x, decoded):  
    xent_loss = K.sum(K.sum(objectives.binary_crossentropy(x ,decoded),axis=-1),axis=-1)
    return xent_loss 


vae = Model(inputs=input_img, outputs=decoded)
vae.compile(optimizer='rmsprop', loss=vae_loss)

In [6]:
vae.fit(normalized_data[:1300], original_label[:1300],
        shuffle=True,
        epochs=100,
        batch_size=batch_size,
        validation_data=(normalized_data[1300:1350], original_label[1300:1350]),callbacks=[EarlyStopping])

Train on 1300 samples, validate on 50 samples
Epoch 1/100
1300/1300 [==============================] - 38s - loss: 12931.1377 - val_loss: 5620.8394
Epoch 2/100
1300/1300 [==============================] - 33s - loss: 10955.8276 - val_loss: 3950.4828
Epoch 3/100
1300/1300 [==============================] - 33s - loss: 11322.8373 - val_loss: 5223.6002
Epoch 4/100
1300/1300 [==============================] - 33s - loss: 10507.9503 - val_loss: 3944.9378
Epoch 5/100
1300/1300 [==============================] - 33s - loss: 10124.4535 - val_loss: 3219.0539
Epoch 6/100
1300/1300 [==============================] - 33s - loss: 9329.1642 - val_loss: 3369.1340
Epoch 7/100
1300/1300 [==============================] - 33s - loss: 9051.2742 - val_loss: 3341.8374
Epoch 8/100
1300/1300 [==============================] - 33s - loss: 8385.7387 - val_loss: 3811.7684
Epoch 9/100
1300/1300 [==============================] - 33s - loss: 8234.7338 - val_loss: 4025.3139
Epoch 10/100
1300/1300 [================